# Imports

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Data

In [44]:
df = pd.read_csv("data/train.csv")

targets = ["wip", "investissement", "satisfaction"]

# 7595 colonnes

# Preprocessing

#### On retire les colonnes 

In [45]:
# unique values

unique_cols = df.columns[df.nunique() <= 1].tolist()

df = df.drop(columns=unique_cols)

In [46]:
# colonne identiques

duplicate_mask = df.T.duplicated()
duplicate_cols = df.columns[duplicate_mask].tolist()

df = df.loc[:, ~duplicate_mask]

In [47]:
# imbalanced columns
        
threshold = 0.97

dominant_ratio = df.apply(
    lambda col: col.value_counts(normalize=True, dropna=False).iloc[0]
)

imbalanced_cols = dominant_ratio[dominant_ratio >= threshold].index.tolist()

df = df.drop(columns=imbalanced_cols)

In [48]:
len(unique_cols)

1484

In [49]:
len(duplicate_cols)

3003

In [50]:
len(imbalanced_cols)

373

In [51]:
df.to_parquet("data/train_V1.parquet")

# Simplification de model

In [52]:
df = pd.read_parquet("data/train_V1.parquet")

n_bins = 10

satisfaction_bins = pd.qcut(
    df["satisfaction"],
    q=n_bins,
    duplicates="drop"
)

df_sample, _ = train_test_split(
    df,
    train_size=0.2,
    random_state=42,
    stratify=satisfaction_bins
)

In [53]:
df_sample

,id,demand_1_0,demand_1_1,demand_3_0,demand_3_1,demand_3_6,demand_4_0,demand_4_2,demand_5_0,demand_5_2,...,param_year0_f16,param_year1_f14,param_year1_f15,param_year1_f16,param_year2_f14,param_year2_f15,param_year2_f16,wip,investissement,satisfaction
16657,16657,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,14.0,30.0,7.0,16.0,30.0,7.0,14.0,2.892445e+07,1125000.0,0.883289
33982,33982,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,24.0,21.0,6.0,24.0,21.0,6.0,24.0,3.346600e+07,1000000.0,0.352099
44478,44478,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,20.0,25.0,12.0,28.0,18.0,18.0,20.0,3.660383e+07,875000.0,0.876640
24762,24762,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,20.0,14.0,12.0,20.0,25.0,12.0,20.0,1.852423e+07,875000.0,0.554435
37180,37180,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,19.0,24.0,6.0,19.0,24.0,6.0,19.0,3.376494e+07,1000000.0,0.952128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64082,64082,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,34.0,22.0,9.0,17.0,22.0,12.0,15.0,2.882004e+07,1250000.0,0.918098
47702,47702,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,26.0,24.0,5.0,15.0,24.0,5.0,16.0,3.153273e+07,1125000.0,0.789337
5847,5847,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,10.0,20.0,12.0,10.0,14.0,12.0,10.0,3.259112e+07,875000.0,0.845144
36816,36816,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,16.0,24.0,6.0,16.0,24.0,6.0,16.0,2.577275e+07,1000000.0,0.694205


# model

In [58]:
# df = df_sample

df = pd.read_parquet("data/train_V1.parquet")

X = df.drop(columns=targets)
y = df[targets]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
pred = model.predict(X_test)

print(mean_absolute_error(y_test, pred))

514917.05877970095


In [59]:
X_train = df.drop(columns=targets)
y_train = df[targets]

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
pred = model.predict(X_test)

print(mean_absolute_error(y_test, pred))

182858.02137678536


# Metric

In [60]:
# prédictions -> DataFrame
df_pred = pd.DataFrame(pred, columns=targets, index=y_test.index)

# vraies valeurs
df_true = y_test.copy()

# erreur absolue par KPI
df_error = (df_true - df_pred).abs()
df_error.columns = [f"{col}_error" for col in targets]

# tableau final
df_results = pd.concat([df_true, df_pred.add_suffix("_pred"), df_error], axis=1)

df_results.head()

,wip,investissement,satisfaction,wip_pred,investissement_pred,satisfaction_pred,wip_error,investissement_error,satisfaction_error
76828,2.916826e+07,875000.0,0.953826,2.880639e+07,888750.0,0.938813,3.618692e+05,13750.0,0.015013
73881,2.997447e+07,1250000.0,0.761158,2.981156e+07,1222500.0,0.776136,1.629100e+05,27500.0,0.014978
2009,3.564799e+07,875000.0,0.846764,3.334808e+07,878750.0,0.834303,2.299911e+06,3750.0,0.012461
26610,2.022953e+07,1125000.0,0.534519,2.292901e+07,1102500.0,0.539631,2.699480e+06,22500.0,0.005112
70500,2.836328e+07,1000000.0,0.950798,2.839017e+07,1002500.0,0.943208,2.689267e+04,2500.0,0.007590


In [57]:
mask = (df_results["satisfaction"] - df_results["satisfaction_pred"]).abs() < 0.05

count_ok = mask.sum()
ratio_ok = mask.mean()  

count_ok, ratio_ok

(2494, 0.8105297367565811)